In [1]:
# 1
# Chance Wiese

In [2]:
# 2
import os

path = '/Users/chancewiese/Dropbox/5330corgis'
files = os.listdir(path)

for file in files:
    print(file)

pdx623.csv
portlandeventjun22.csv
newreg-062822.csv
Icon
newcorgis-20220619.csv
newcorgis-20220618.csv
signupsjune272022.csv
tacoma-062322.csv


In [3]:
# 3
import pandas as pd
df = pd.read_csv('/Users/chancewiese/Dropbox/5330corgis/pdx623.csv')
print(df)

                            Name Breed Gender  Weight  Age
0  Wendell of the Gnashing Teeth   CWC     NM      32  3.3
1                Trotsky the Dog   Pem     SF      23  5.5
2                CanCan of Beans   Pem     SF      25  4.2
3             Harvey the Younger   PWC      M      22  1.1
4                 Jenni the Shiv   PWC     SF      28  3.0
5           Ogbert the Semi-Wise   Pem     NM      25  4.0
6                Jeff OCorgosaur   PWC      M      27  2.0
7                  Poofy Mudfoot   Pem     SF      23  2.1


In [4]:
# 4
import psycopg2

conn = psycopg2.connect(
    database='corgirace',
    user='dw_chancewiese',
    password='Spikeball2020',
    host='database-1.czsooswggscz.us-east-2.rds.amazonaws.com',
    port='5432'
)

cursor = conn.cursor()

query = "SELECT * FROM ingest;"
df_ingest = pd.read_sql_query(query, conn)


for file in files:
    file_path = os.path.join(path, file)
    file_lower = file.lower()
    if not file.endswith('.csv'):
        print(f"{file_lower} not a csv - skipped.")
    elif file_lower in df_ingest['filename'].values:
        print(f"{file_lower} already ingested - skipped.")
    else:
        print(f"{file_lower} needs to be ingested - ingesting now.")
        df = pd.read_csv(file_path)
        
        # Data Cleaning
        df['Breed'] = df['Breed'].replace(['PWC', 'CWC'], ['Pem', 'Cardi']) # Breed
        df['Gender'] = df['Gender'].replace('Fem', 'F') # Gender
        df['Weight'] = df['Weight'].round().astype(int) # Weight
        df['Age'] = (df['Age'] * 2).round() / 2 # Age
        
        # Load corgi
        for x in df.index:
            cursor.execute(f'''
            INSERT INTO corgi (corgname, breed, gender, weight, age, fromfile)
            VALUES (
                '{df['Name'].loc[x]}',
                '{df['Breed'].loc[x]}',
                '{df['Gender'].loc[x]}',
                '{df['Weight'].loc[x]}',
                '{df['Age'].loc[x]}',
                '{file_lower}');
            ''')
            print(f"Record created for {df['Name'].loc[x]}")

        conn.commit()

        # Load ingest
        cursor.execute(f'''
        INSERT INTO ingest (filename, whendone)
        VALUES ('{file_lower}', current_timestamp);
        ''')

        conn.commit()
        print(f"Ingestion complete for {file_lower}.")

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_31302/1034903121.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ingest = pd.read_sql_query(query, conn)


pdx623.csv needs to be ingested - ingesting now.
Record created for Wendell of the Gnashing Teeth
Record created for Trotsky the Dog
Record created for CanCan of Beans
Record created for Harvey the Younger
Record created for Jenni the Shiv
Record created for Ogbert the Semi-Wise
Record created for Jeff OCorgosaur
Record created for Poofy Mudfoot
Ingestion complete for pdx623.csv.
portlandeventjun22.csv needs to be ingested - ingesting now.
Record created for Amber Waves of Fur
Record created for Kodlak Whitemane
Record created for Floaty Bee
Record created for Bag o Donuts
Record created for Hot Ham and Cheese
Record created for Cruzer the Cruzedog
Record created for Sally the Destroyer
Record created for Elma Redfur
Record created for Kool Thing
Record created for Skitters McDogdog
Record created for Ophelio
Record created for Mr. Stealyoursandwich
Ingestion complete for portlandeventjun22.csv.
newreg-062822.csv needs to be ingested - ingesting now.
Record created for Megalodon
Record

In [5]:
# 5
query = "SELECT * FROM ingest;"
df_verify_ingest = pd.read_sql_query(query, conn)
print("\nVerify ingest table:\n", df_verify_ingest)


Verify ingest table:
                  filename                   whendone
0  newcorgis-20220618.csv 2024-10-07 03:15:29.154882
1              pdx623.csv 2024-10-16 22:30:30.830070
2  portlandeventjun22.csv 2024-10-16 22:30:31.835271
3       newreg-062822.csv 2024-10-16 22:30:32.411593
4  newcorgis-20220619.csv 2024-10-16 22:30:32.987303
5   signupsjune272022.csv 2024-10-16 22:30:33.602819
6       tacoma-062322.csv 2024-10-16 22:30:34.359558


/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_31302/3204024492.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_verify_ingest = pd.read_sql_query(query, conn)


In [13]:
# 6
query = '''
SELECT corgid, corgname, fromfile, DATE(whendone) AS whendone_date
FROM corgi c
JOIN ingest i ON c.fromfile = i.filename
WHERE EXTRACT(YEAR FROM whendone) = EXTRACT(YEAR FROM CURRENT_DATE)
AND EXTRACT(MONTH FROM whendone) = EXTRACT(MONTH FROM CURRENT_DATE)
ORDER BY corgname;
'''
df_verify_corgi = pd.read_sql_query(query, conn, index_col='corgid')
print("\nVerify ingest table:\n", df_verify_corgi)


Verify ingest table:
                              corgname                fromfile whendone_date
corgid                                                                     
43                 Amber Waves of Fur  portlandeventjun22.csv    2024-10-16
46                       Bag o Donuts  portlandeventjun22.csv    2024-10-16
59                Betty Doggy Sprinty       newreg-062822.csv    2024-10-16
74                         Briar Rose       tacoma-062322.csv    2024-10-16
37                    CanCan of Beans              pdx623.csv    2024-10-16
65                         Cashew Nut   signupsjune272022.csv    2024-10-16
73                          Cleopupra       tacoma-062322.csv    2024-10-16
70                     Courtesy Check   signupsjune272022.csv    2024-10-16
48                Cruzer the Cruzedog  portlandeventjun22.csv    2024-10-16
71                             Dogpup       tacoma-062322.csv    2024-10-16
50                        Elma Redfur  portlandeventjun22.csv    

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_31302/3989507299.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_verify_corgi = pd.read_sql_query(query, conn, index_col='corgid')
